In [73]:
import csv
import pandas as pd
import os
from tqdm import tqdm
from fuzzywuzzy import fuzz
import nltk
from nltk.tokenize import word_tokenize

In [6]:
file_path = '/u/spa-d4/grad/mfe261/Projects/MobileConvRec/data/master_app_data_V1_true.csv'
data = pd.read_csv(file_path)

In [8]:
print(len(data))

1730


In [11]:
num_apps = data['app_package'].nunique()
num_categories = data['app_category'].nunique()

print("Number of unique apps: ", num_apps)
print("Number of unique categories: ", num_categories)

Number of unique apps:  1730
Number of unique categories:  45


In [9]:
# Load the second CSV file
file_path_2 = '/u/spa-d4/grad/mfe261/Projects/MobileConvRec/data/final_review_data_update_V2.csv'
data_2 = pd.read_csv(file_path_2)

In [14]:
data_2.columns

Index(['reviewId', 'review', 'app_package', 'User Interface Design',
       'Navigation', 'Accessibility', 'Customization', 'Functionality',
       'Performance', 'Responsiveness', 'Security', 'Privacy', 'Permissions',
       'Data Collection', 'Data Sharing', 'Updates', 'Customer support',
       'Reviews and ratings', 'Developer', 'Price', 'In-app purchases',
       'Advertisement Frequency', 'Battery/power Drainage', 'aspect_count',
       'formated_date', 'rating', 'votes', 'uid', 'app_name'],
      dtype='object')

In [10]:
print(len(data_2))

12340


In [15]:
# Get the number of unique users from the 'uid' column
num_unique_users = data_2['uid'].nunique()

print("Number of unique users:", num_unique_users)

Number of unique users: 11863


In [56]:
# List all files in the folder
training_files = os.listdir("data/dataset_v2/training")
testing_files = os.listdir("data/dataset_v2/testing")
validation_files = os.listdir("data/dataset_v2/validation")
num_dialog = len(training_files) + len(testing_files) + len(validation_files)
print("Total number of dialogs:", num_dialog)

Total number of dialogs: 12562


In [22]:
def is_approximate_substring(substring, string, threshold=70):
    for i in range(len(string) - len(substring) + 1):
        window = string[i:i+len(substring)]
        similarity_ratio = fuzz.ratio(substring, window)
        if similarity_ratio >= threshold:
            return True
    return False

In [120]:
text = """
User's Previous Interactions
No History Available

Recommended App Name: Firefox Fast & Private Browser | Package Name: org.mozilla.firefox | Date: 16-11-2018

Negative Recommended App Name: Opera Mini - fast web browser | Package Name: com.opera.mini.native
Negative Recommended App Name: Chrome Dev | Package Name: com.chrome.dev
Negative Recommended App Name: Opera browser beta | Package Name: com.opera.browser.beta

COMPUTER: Hi there! How can I help you today?
"""

count = text.count("Negative Recommended App Name:")
print(f"The phrase 'Negative Recommended App Name:' appears {count} times.")


The phrase 'Negative Recommended App Name:' appears 3 times.


In [121]:
conversation_training_path = "data/dataset_v2/training"

user_id = []
previous_interactions = []
previous_interactions_array = []
count_neg_recommended_all = []
recommended_app_name = []
turns = []
recommend_indexes = []


# List all files in the folder
files = os.listdir(conversation_training_path)

# Loop through each file in the directory
for filename in tqdm(files):
    # Construct the full file path
    file_path = os.path.join(conversation_training_path, filename)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path, 'r') as file:
            # Read the content of the file
            file_content = file.read().lower()
            
            # finding the user id
            index_1 = file_content.find("user's previous interactions")
            user_id.append(file_content[9:index_1].rstrip('\n'))
            
            # finding the User's Previous Interactions
            index_2 = file_content.find("recommended app name:")
            previous_interactions_arr = file_content[index_1+29:index_2].rstrip('\n').split("app name:")
            previous_interactions_arr_filtered = []
            for previous_interaction in previous_interactions_arr[1:]:
                previous_interactions_arr_filtered.append(previous_interaction[:previous_interaction.find(" | ")])
            if len(previous_interactions_arr_filtered) > 0:
                previous_interactions.append(",".join(previous_interactions_arr_filtered))
                previous_interactions_array.append(previous_interactions_arr_filtered)
            else:
                previous_interactions.append(None)
                previous_interactions_array.append([])
            
            # finding recommended app name
            index_3 = file_content[index_2:].find("package name")
            recommended = file_content[index_2+22:index_2+index_3-3].rstrip('\n')
            recommended_app_name.append(recommended)
            
            # finding the number of times negative recommended apps are mentioned in the file
            count_neg_recommended = file_content.count("negative recommended app name:")
            count_neg_recommended_all.append(count_neg_recommended)
            
            # finding each turns
            dialog_turns = []
            dialog_index = 0
            COMPUTER_index = file_content.find("computer:")
            file_content = file_content[COMPUTER_index:]
            found_recommender = False
            while True:
                HUMAN_index = file_content.find("human:")
                if HUMAN_index == -1:
                    break
                turn = file_content[:HUMAN_index].rstrip('\n') # computer dialog
                if (recommended in turn) and not found_recommender:
                    recommend_indexes.append(dialog_index)
                    found_recommender = True
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[HUMAN_index:]
                
                COMPUTER_index = file_content.find("computer:")
                turn = file_content[:COMPUTER_index].rstrip('\n') # human dialog
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[COMPUTER_index:]
                
            if not found_recommender: # approximately finding the recommender turn
                for i, dialog_turn in enumerate(dialog_turns):
                    if is_approximate_substring(recommended, dialog_turn):
                        recommend_indexes.append(i)
                        found_recommender = True
                        break
                    
            if not found_recommender:
                recommend_indexes.append(-1)
                        
            turns.append(dialog_turns)

print(len(user_id))
print(len(previous_interactions))
print(len(recommended_app_name))
print(len(recommend_indexes))
df_recommender_train = pd.DataFrame({"user_id": user_id, "previous_interactions":previous_interactions, "previous_interactions_array": previous_interactions_array, "recommended_app_name":recommended_app_name, "count_neg_recommended_all": count_neg_recommended_all, "turns": turns, "recommend_indexes":recommend_indexes})
print(f"\nnumber of rows: {len(df_recommender_train)}")

100%|██████████| 8720/8720 [00:26<00:00, 329.76it/s]

8720
8720
8720
8720

number of rows: 8720


In [24]:
# df_recommender_train = df_recommender_train[(df_recommender_train["recommend_indexes"] != -1) & (df_recommender_train["turns"].apply(lambda x: len(x) > 0))]

In [122]:
conversation_validation_path = "data/dataset_v2/validation"

user_id = []
previous_interactions = []
previous_interactions_array = []
count_neg_recommended_all = []
recommended_app_name = []
turns = []
recommend_indexes = []


# List all files in the folder
files = os.listdir(conversation_validation_path)

# Loop through each file in the directory
for filename in tqdm(files):
    # Construct the full file path
    file_path = os.path.join(conversation_validation_path, filename)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path, 'r') as file:
            # Read the content of the file
            file_content = file.read().lower()
            
            # finding the user id
            index_1 = file_content.find("user's previous interactions")
            user_id.append(file_content[9:index_1].rstrip('\n'))
            
            # finding the User's Previous Interactions
            index_2 = file_content.find("recommended app name:")
            previous_interactions_arr = file_content[index_1+29:index_2].rstrip('\n').split("app name:")
            previous_interactions_arr_filtered = []
            for previous_interaction in previous_interactions_arr[1:]:
                previous_interactions_arr_filtered.append(previous_interaction[:previous_interaction.find(" | ")])
            if len(previous_interactions_arr_filtered) > 0:
                previous_interactions.append(",".join(previous_interactions_arr_filtered))
                previous_interactions_array.append(previous_interactions_arr_filtered)
            else:
                previous_interactions.append(None)
                previous_interactions_array.append([])
            
            # finding recommended app name
            index_3 = file_content[index_2:].find("package name")
            recommended = file_content[index_2+22:index_2+index_3-3].rstrip('\n')
            recommended_app_name.append(recommended)
            
            # finding the number of times negative recommended apps are mentioned in the file
            count_neg_recommended = file_content.count("negative recommended app name:")
            count_neg_recommended_all.append(count_neg_recommended)
            
            # finding each turns
            dialog_turns = []
            dialog_index = 0
            COMPUTER_index = file_content.find("computer:")
            file_content = file_content[COMPUTER_index:]
            found_recommender = False
            while True:
                HUMAN_index = file_content.find("human:")
                if HUMAN_index == -1:
                    break
                turn = file_content[:HUMAN_index].rstrip('\n') # computer dialog
                if (recommended in turn) and not found_recommender:
                    recommend_indexes.append(dialog_index)
                    found_recommender = True
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[HUMAN_index:]
                
                COMPUTER_index = file_content.find("computer:")
                turn = file_content[:COMPUTER_index].rstrip('\n') # human dialog
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[COMPUTER_index:]
                
            if not found_recommender: # approximately finding the recommender turn
                for i, dialog_turn in enumerate(dialog_turns):
                    if is_approximate_substring(recommended, dialog_turn):
                        recommend_indexes.append(i)
                        found_recommender = True
                        break
                    
            if not found_recommender:
                recommend_indexes.append(-1)
                        
            turns.append(dialog_turns)

print(len(user_id))
print(len(previous_interactions))
print(len(recommended_app_name))
print(len(recommend_indexes))
df_recommender_validation = pd.DataFrame({"user_id": user_id, "previous_interactions":previous_interactions, "previous_interactions_array": previous_interactions_array, "recommended_app_name":recommended_app_name, "count_neg_recommended_all": count_neg_recommended_all, "turns": turns, "recommend_indexes":recommend_indexes})
print(f"\nnumber of rows: {len(df_recommender_validation)}")

100%|██████████| 1285/1285 [00:04<00:00, 276.50it/s]

1285
1285
1285
1285

number of rows: 1285


In [123]:
# df_recommender_validation = df_recommender_validation[(df_recommender_validation["recommend_indexes"] != -1) & (df_recommender_validation["turns"].apply(lambda x: len(x) > 0))]

In [124]:
conversation_test_path = "data/dataset_v2/testing"

user_id = []
previous_interactions = []
previous_interactions_array = []
count_neg_recommended_all = []
recommended_app_name = []
turns = []
recommend_indexes = []


# List all files in the folder
files = os.listdir(conversation_test_path)

# Loop through each file in the directory
for filename in tqdm(files):
    # Construct the full file path
    file_path = os.path.join(conversation_test_path, filename)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path, 'r') as file:
            # Read the content of the file
            file_content = file.read().lower()
            
            # finding the user id
            index_1 = file_content.find("user's previous interactions")
            user_id.append(file_content[9:index_1].rstrip('\n'))
            
            # finding the User's Previous Interactions
            index_2 = file_content.find("recommended app name:")
            previous_interactions_arr = file_content[index_1+29:index_2].rstrip('\n').split("app name:")
            previous_interactions_arr_filtered = []
            for previous_interaction in previous_interactions_arr[1:]:
                previous_interactions_arr_filtered.append(previous_interaction[:previous_interaction.find(" | ")])
            if len(previous_interactions_arr_filtered) > 0:
                previous_interactions.append(",".join(previous_interactions_arr_filtered))
                previous_interactions_array.append(previous_interactions_arr_filtered)
            else:
                previous_interactions.append(None)
                previous_interactions_array.append([])
            
            # finding recommended app name
            index_3 = file_content[index_2:].find("package name")
            recommended = file_content[index_2+22:index_2+index_3-3].rstrip('\n')
            recommended_app_name.append(recommended)
            
            # finding the number of times negative recommended apps are mentioned in the file
            count_neg_recommended = file_content.count("negative recommended app name:")
            count_neg_recommended_all.append(count_neg_recommended)
            
            # finding each turns
            dialog_turns = []
            dialog_index = 0
            COMPUTER_index = file_content.find("computer:")
            file_content = file_content[COMPUTER_index:]
            found_recommender = False
            while True:
                HUMAN_index = file_content.find("human:")
                if HUMAN_index == -1:
                    break
                turn = file_content[:HUMAN_index].rstrip('\n') # computer dialog
                if (recommended in turn) and not found_recommender:
                    recommend_indexes.append(dialog_index)
                    found_recommender = True
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[HUMAN_index:]
                
                COMPUTER_index = file_content.find("computer:")
                turn = file_content[:COMPUTER_index].rstrip('\n') # human dialog
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[COMPUTER_index:]
                
            if not found_recommender: # approximately finding the recommender turn
                for i, dialog_turn in enumerate(dialog_turns):
                    if is_approximate_substring(recommended, dialog_turn):
                        recommend_indexes.append(i)
                        found_recommender = True
                        break
                    
            if not found_recommender:
                recommend_indexes.append(-1)
                        
            turns.append(dialog_turns)

print(len(user_id))
print(len(previous_interactions))
print(len(recommended_app_name))
print(len(recommend_indexes))
df_recommender_test = pd.DataFrame({"user_id": user_id, "previous_interactions":previous_interactions, "previous_interactions_array": previous_interactions_array, "recommended_app_name":recommended_app_name, "count_neg_recommended_all": count_neg_recommended_all, "turns": turns, "recommend_indexes":recommend_indexes})
print(f"\nnumber of rows: {len(df_recommender_test)}")

  0%|          | 0/2557 [00:00<?, ?it/s]

100%|██████████| 2557/2557 [00:08<00:00, 318.40it/s]

2557
2557
2557
2557

number of rows: 2557


In [29]:
# df_recommender_test = df_recommender_test[(df_recommender_test["recommend_indexes"] != -1) & (df_recommender_test["turns"].apply(lambda x: len(x) > 0))]

In [125]:
df_recommender_train["turns"]

0       [computer: hello! how can i assist you today?,...
1       [computer: hi there! what are you looking for ...
2       [computer: are you looking for apps that are e...
3       [computer: hi! how can i help you today?, huma...
4       [computer: hey there! what can i help you with...
                              ...                        
8715    [computer: hi there! how can i help you today?...
8716    [computer: hey there! how can i help you today...
8717    [computer: sure thing! what kinds of zombie hu...
8718    [computer: hi there! how can i help you today?...
8719    [computer: hi there! what brings you here toda...
Name: turns, Length: 8720, dtype: object

In [126]:
df_recommender_validation["turns"]

0       [computer: hi there! how can i help you today?...
1       [computer: hi there! what brings you here toda...
2       [computer: hey there! how can i help you today...
3       [computer: hey there! what are you looking for...
4       [computer: hello! how can i help you today?, h...
                              ...                        
1280    [computer: hello! how can i help you today?, h...
1281    [computer: hi there! how can i help you today?...
1282    [computer: hey there! what can i help you find...
1283    [computer: hi there! what are you looking for ...
1284    [computer: hey there! what brings you here tod...
Name: turns, Length: 1285, dtype: object

In [127]:
df_recommender_test["turns"]

0       [computer: hey there! how can i help you today...
1       [computer: hello! how can i help you today?, h...
2       [computer: hi there! how can i help you today?...
3       [computer: hi! what can i help you with today?...
4       [computer: hello! how can i help you today?, h...
                              ...                        
2552    [computer: hi there! how can i help you today?...
2553    [computer: hi there! how can i assist you toda...
2554    [computer: hey! how can i help you today?, hum...
2555    [computer: hi there! how can i help you today?...
2556    [computer: hi! how can i help you today?, huma...
Name: turns, Length: 2557, dtype: object

In [129]:
len(df_recommender_validation["turns"][0])

22

In [130]:
len(df_recommender_train["turns"]) + len(df_recommender_validation["turns"]) + len(df_recommender_test["turns"])

12562

In [131]:
num_dialog

12562

In [165]:
total_turns = 0

for turn in df_recommender_train["turns"]:
    total_turns += len(turn)
    
for turn in df_recommender_validation["turns"]:
    total_turns += len(turn)

for turn in df_recommender_test["turns"]:
    total_turns += len(turn)

total_turns = int(total_turns / 2) # both computer and human are considered to be one turn

print("Number of unique turns:", total_turns)


Number of unique turns: 148166


In [133]:
df_recommender_test["turns"]

0       [computer: hey there! how can i help you today...
1       [computer: hello! how can i help you today?, h...
2       [computer: hi there! how can i help you today?...
3       [computer: hi! what can i help you with today?...
4       [computer: hello! how can i help you today?, h...
                              ...                        
2552    [computer: hi there! how can i help you today?...
2553    [computer: hi there! how can i assist you toda...
2554    [computer: hey! how can i help you today?, hum...
2555    [computer: hi there! how can i help you today?...
2556    [computer: hi! how can i help you today?, huma...
Name: turns, Length: 2557, dtype: object

In [134]:
num_dialog

12562

In [135]:
print("Average turns per dialog:", total_turns / num_dialog)

Average turns per dialog: 23.589555803216047


In [136]:
# Download the necessary data for tokenization and stopwords
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     /u/spa-d4/grad/mfe261/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /u/spa-d4/grad/mfe261/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [137]:
def count_words(sentence):
    """
    Count the number of words in a sentence, excluding punctuation.

    Parameters:
    sentence (str): The input sentence.

    Returns:
    int: The number of words in the sentence.
    """
    # Tokenize the sentence into words
    words = word_tokenize(sentence)

    # Remove punctuation from the list of words
    words = [word for word in words if word.isalnum()]

    # Count the number of words
    word_count = len(words)

    return word_count

# Sample usage
sentence = "I am kinda shy, but I am trying to improve myself."
print("The number of words in the sentence is:", count_words(sentence))

The number of words in the sentence is: 11


In [138]:
turn

['computer: hi! how can i help you today?',
 "human: i'm looking for offline shooting games for my android device.",
 "computer: that's cool! do you want games with easy controls?",
 'human: yes, i prefer games with easy controls.',
 'computer: what rating are you looking for in the apps?',
 'human: i like apps with at least a 4.7 star rating.',
 'computer: are you looking for free apps, or are you open to paying for extra features?',
 "human: i'd prefer free apps.",
 'computer: are you okay with apps that have ads?',
 'human: a few ads are okay with me.',
 'computer: i recommend trying out the stacky dash app.',
 'human: can you tell me more about the stacky dash app?',
 'computer: it has relaxing gameplay, but some users have mentioned too many ads.',
 "human: i'll pass on that app. do you have any other suggestions?",
 'computer: how about trying the hills of steel app?',
 'human: tell me more about the hills of steel app.',
 'computer: it has multiplayer features and customizable t

In [139]:
total_words = 0

for turn in tqdm(df_recommender_train["turns"]):
    for each_turn in turn:
        if "computer" in each_turn:
            processed_turn = each_turn[9:]
        elif "human" in each_turn:
            processed_turn = each_turn[6:]
        else:
            print("sth is wrong")
        total_words += count_words(processed_turn)
    
for turn in tqdm(df_recommender_validation["turns"]):
    for each_turn in turn:
        if "computer" in each_turn:
            processed_turn = each_turn[9:]
        elif "human" in each_turn:
            processed_turn = each_turn[6:]
        else:
            print("sth is wrong")
        total_words += count_words(processed_turn)

for turn in tqdm(df_recommender_test["turns"]):
    for each_turn in turn:
        if "computer" in each_turn:
            processed_turn = each_turn[9:]
        elif "human" in each_turn:
            processed_turn = each_turn[6:]
        else:
            print("sth is wrong")
        total_words += count_words(processed_turn)

print("Average words per turns:", total_words / total_turns)


100%|██████████| 2557/2557 [00:05<00:00, 477.28it/s]

Average words per turns: 9.701429477747931


In [112]:
df_recommender_test['previous_interactions_array'][35]

[' city island 5 - building sim',
 ' truck simulator 2018 : europe',
 ' ringid- live stream, live tv and online shopping',
 ' second galaxy',
 ' kim kardashian: hollywood',
 ' my little pony: harmony quest',
 ' parchisi star online',
 ' property brothers home design',
 ' voice access',
 ' solitaire tripeaks: farm and family']

In [141]:
total_previous_interactions = 0

for previous_interactions in tqdm(df_recommender_train["previous_interactions_array"]):
    total_previous_interactions += len(previous_interactions)
    
for previous_interactions in tqdm(df_recommender_validation["previous_interactions_array"]):
    total_previous_interactions += len(previous_interactions)

for previous_interactions in tqdm(df_recommender_test["previous_interactions_array"]):
    total_previous_interactions += len(previous_interactions)

print("Number of previous interactions:", total_previous_interactions)
print("Average number of previous interactions:", total_previous_interactions / num_dialog)

100%|██████████| 2557/2557 [00:00<00:00, 1808572.57it/s]

Number of previous interactions: 88921
Average number of previous interactions: 7.07857029135488


In [163]:
# Find the length of each array
lengths = df_recommender_test["previous_interactions_array"].apply(len)

lengths.value_counts()

previous_interactions_array
1     592
10    396
2     339
3     260
0     184
4     181
5     157
6     145
7     113
8     107
9      83
Name: count, dtype: int64

In [145]:
total_neg_recommended = 0

for count_neg_recommended in tqdm(df_recommender_train["count_neg_recommended_all"]):
    total_neg_recommended += count_neg_recommended
    
for count_neg_recommended in tqdm(df_recommender_validation["count_neg_recommended_all"]):
    total_neg_recommended += count_neg_recommended

for count_neg_recommended in tqdm(df_recommender_test["count_neg_recommended_all"]):
    total_neg_recommended += count_neg_recommended

print("Number of negative recommendations:", total_neg_recommended)
print("Average number of negative recommendations per dialog:", total_neg_recommended / num_dialog)

100%|██████████| 2557/2557 [00:00<00:00, 2045553.18it/s]

Number of negative recommendations: 28365
Average number of negative recommendations per dialog: 2.258000318420634
